In [10]:
import pandas as pd

df = pd.read_csv("~/work/dataset/RC15/yoochoose-buys.dat", sep=",", header=None)
df

,0,1,2,3,4
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1
...,...,...,...,...,...
1150748,11368701,2014-09-26T07:52:51.357Z,214849809,554,2
1150749,11368691,2014-09-25T09:37:44.206Z,214700002,6806,5
1150750,11523941,2014-09-25T06:14:47.965Z,214578011,14556,1
1150751,11423202,2014-09-26T18:49:34.024Z,214849164,1046,1


In [5]:
from flax import linen as nn
from jax import numpy as jnp
import numpy as np
import jax
import flax

class NormalGRU(nn.Module):
    hidden_dim: int
    dropout_rate : float
    @nn.compact
    def __call__(
        self, 
        q:jnp.ndarray, # (batch_size, seq_len), アイテム
    ):
        
        carry = self.param("init_carry", lambda rng, shape: jnp.zeros(shape), self.hidden_dim)
        
        for i in range(q.shape[1]):
            carry, out = nn.GRUCell()(carry, q[:, i, :])
        
        #output = nn.Dense(1)(carry)
        return carry, out

In [6]:
key = jax.random.PRNGKey(0)
rng, key = jax.random.split(key)
a = jax.random.normal(key, (5, 2, 4))
a

DeviceArray([[[-6.6552585e-01, -8.0536044e-01, -2.7475157e+00,
               -4.7615182e-01],
              [ 6.3546067e-01,  2.3215576e-03, -1.0808130e-01,
               -1.0878880e+00]],

             [[-6.9108629e-01,  9.8934972e-01,  7.4207038e-01,
                6.0069878e-02],
              [ 3.4648937e-01, -2.7497369e-01, -1.9377939e+00,
               -9.3766487e-01]],

             [[-6.2804383e-01,  1.1748193e-02,  2.0391178e+00,
                4.1310093e-01],
              [ 6.7216545e-01, -1.7155396e+00,  3.4572795e-02,
                1.3146477e+00]],

             [[ 6.0303673e-02,  3.4236613e-01, -6.0134846e-01,
               -7.7926105e-01],
              [-1.0060278e+00, -7.3032057e-01,  5.8969802e-01,
                1.6912601e+00]],

             [[ 8.4735954e-01, -2.9250801e-01, -2.8486779e-01,
               -9.7763616e-01],
              [ 1.7288361e+00, -1.4076442e+00,  5.6083947e-01,
               -1.3720120e+00]]], dtype=float32)

In [7]:
rng, key = jax.random.split(rng)
model = NormalGRU(10, 0.2)
params = model.init(key, a)["params"]

In [8]:
model.apply({"params":params}, a)

(DeviceArray([[ 0.24692631,  0.49412495,  0.4746471 , -0.32867706,
               -0.13564254, -0.15415218, -0.19948936,  0.11361546,
               -0.7855797 , -0.1430701 ],
              [-0.3002967 , -0.512416  ,  0.10101941, -0.24048063,
                0.23212117, -0.2228501 ,  0.330711  , -0.21408081,
               -0.09280246, -0.19987902],
              [-0.02015809, -0.5048353 , -0.82930255,  0.62537503,
                0.2861167 , -0.0401026 , -0.5663145 , -0.3203154 ,
                0.09992053,  0.9088545 ],
              [-0.21178085, -0.35982224, -0.38087508, -0.0109929 ,
               -0.64698297,  0.45345804, -0.79816103, -0.25303644,
                0.23362584,  0.53609204],
              [ 0.2844317 ,  0.18422572, -0.04782872, -0.3885925 ,
                0.4353066 , -0.2508729 ,  0.11873263, -0.6231649 ,
               -0.5470435 ,  0.631553  ]], dtype=float32),
 DeviceArray([[ 0.24692631,  0.49412495,  0.4746471 , -0.32867706,
               -0.13564254, -0.15415

In [9]:
model.param.ini

<bound method Module.param of NormalGRU(
    # attributes
    hidden_dim = 10
    dropout_rate = 0.2
)>